[![Open In
Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/CU-Denver-MathStats-OER/Statistical-Theory/blob/main/Chap5/17-Bootstrap-Confidence-Int.ipynb)
 

In our [work with bootstrap distributions](16-Bootstrap-Dist.qmd), we
explored how we can use data from one random sample picked from a
population with unknown parameters to construct an approximation for a
sampling distribution. From a bootstrap distribution, we can define a
new estimator:

-   We defined the <span style="color:dodgerblue">estimator
    $\hat{\theta}_{\rm{boot}}$</span> as the mean (expected) value of
    the bootstrap distribution.
-   We use the <span style="color:dodgerblue">bootstrap standard
    error</span> to estimate the variability or uncertainty in estimates
    due to the randomness of sampling.
-   In the case of estimating a population mean $\mu$ with a bootstrap
    estimator, we defined the <span style="color:dodgerblue">bootstrap
    estimate of bias</span> as

$${\color{dodgerblue}{\mbox{Bias}_{\rm{boot}} \big( \hat{\mu}_{\rm{boot}} \big) = \hat{\mu}_{\rm{boot}} - \bar{x}}}.$$

In this section we will discuss how we can use the bootstrap standard
error to account for the uncertainty in our estimate and to incorporate
that uncertainty in our final estimate.

# <a name="17nyc">Case Study: Ozone Concentration in New York City</a>

------------------------------------------------------------------------

<figure>
<img src="https://www-f.nescaum.org/documents/listos/AQI_Map_170518.png"
style="width:65.0%" data-fig-align="left"
data-fig-alt="Map of Air Quality in Northeastern US"
alt="Credit: Northeast States for Coordinated Air Use Management (NESCAUM)" />
<figcaption aria-hidden="true">Credit: <a
href="https://www-f.nescaum.org/documents/listos">Northeast States for
Coordinated Air Use Management (NESCAUM)</a></figcaption>
</figure>

According to the Unites States Environmental Protection Agency (EPA)[1]:

> Ozone (O3) is a highly reactive gas composed of three oxygen atoms. It
> is both a natural and a man-made product that occurs in the Earth’s
> upper atmosphere (the stratosphere) and lower atmosphere (the
> troposphere). Depending on where it is in the atmosphere, ozone
> affects life on Earth in either good or bad ways.

When the ozone concentration is high, respiratory illnesses such as
asthma, pneumonia, and bronchitis can become exacerbated. While the
effects of short-term exposure to high ozone concentration are
reversible, long-term exposure may not be reversible. The EPA sets Ozone
National Ambient Air Quality Standards (NAAQS)[2]. “The existing primary
and secondary standards, established in 2015, are 0.070 parts per
million (ppm)”, or equivalently 70 parts per billion (ppb).

We will begin today’s work with bootstrap distributions by investigating
the following question:

> <span style="color:dodgerblue">Is the population of New York City
> experiencing long-term exposure to unhealthy levels of ozone?</span>

## <a name="17load-nyc">Loading NYC Ozone Data</a>

------------------------------------------------------------------------

It is likely you do not have the package `lattice` installed. You will
need to first install the `lattice` package.

-   Go to the R Console window.
-   Run the command `> install.packages("lattice")`.

You will only need to run the `install.package()` command one time. You
can now access `lattice` anytime you like! However, you will need to run
the command `library(lattice)` during any R session in which you want to
access data from the `lattice` package. **Be sure you have first
installed the `lattice` package before executing the code cell below.**

[1] <https://www.epa.gov/ozone-pollution-and-your-patients-health/what-ozone>
accessed June 26, 2023

[2] [Ozone Air Quality Standards acccessed from EPA June 26,
2023](https://www.epa.gov/naaqs/ozone-o3-air-quality-standards)

In [ ]:
# be sure you have already installed the lattice package
library(lattice)  # loading lattice package

### <a name="17store-nyc">Summarizing and Storing the Data</a>

------------------------------------------------------------------------

In the code cell below we summarize a data frame[1] named
`environmental` from the `lattice` package and store the ozone
concentration data `environmental$ozone` to a vector named `nyc.oz`.

[1] Bruntz, Cleveland, Kleiner, and Warner. (1974). “The Dependence of
Ambient Ozone on Solar Radiation, Wind, Temperature, and Mixing Height”.
In *Symposium on Atmospheric Diffusion and Air Pollution*. American
Meteorological Society, Boston.

In [ ]:
summary(environmental)  # numerical summary of each variable
nyc.oz <- environmental$ozone  # store ozone data to a vector

## <a name="17q1">Question 1</a>

------------------------------------------------------------------------

We will use the sample data stored in `nyc.oz` to construct a bootstrap
distribution that we can use to make predictions about the ozone
concentration for the population of all times in New York City. Answer
the questions below to get acquainted with the sample data.

### <a name="17q1a">Question 1a</a>

------------------------------------------------------------------------

How many observations are in the sample stored in `nyc.oz`? Describe the
shape of the data in `nyc.oz`.

#### <a name="17sol1a">Solution to Question 1a</a>

------------------------------------------------------------------------

<br>  
<br>  
<br>

### <a name="17q1b">Question 1b</a>

------------------------------------------------------------------------

Based on the sample data in `nyc.oz`, give an estimate for the mean
ozone concentration in New York City (over all days and times).

#### <a name="17sol1b">Solution to Question 1b</a>

------------------------------------------------------------------------

<br>  
<br>  
<br>

# <a name="17boot-prop">Bootstrap Distribution for a Proportion</a>

------------------------------------------------------------------------

The sample mean for ozone concentration is below the 70 ppb limit.
However, from the plot in \[Question 1a\], we see there are number of
observations in the sample with an ozone concentration over 70 ppb.

In [ ]:
sum(nyc.oz > 70)

> <span style="color:dodgerblue">What **proportion of time** in NYC is
> the ozone concentration greater than 70 ppb?</span>

The sample proportion of observations with an ozone concentration
greater than 70 ppb is $24/111 \approx 0.2162$.

We calculate the sample proportion in two different ways below. In both
methods, we use the logical test `nyc.oz > 70` to help count the number
of observations greater than 70 ppb.

1.  Using a logical test in a `sum()` command and dividing by the sample
    size `n`.
2.  Using a logical test in a `mean()` command both sums and divides by
    $n$ at once.

In [ ]:
# method 1
n <- length(nyc.oz)  # number of observations in sample
sum(nyc.oz > 70) / n  # sample proportion over 70 ppb

In [ ]:
# method 2
mean(nyc.oz > 70)  # sample proportion over 70 ppb

Both methods are equivalent and we see that

$$\hat{p} = 0.2162 = 21.62 \%.$$

Using the [plug-in principle](16-Bootstrap-Dist.qmd#sec-plugin), a
reasonable estimate for the proportion of all time that the ozone
concentration in NYC is over 70 ppb (we denote the population proportion
$p$) is

$$ p \approx \hat{p} = 0.2612.$$

**How certain can we be in this estimate?** Most likely, the sample
proportion is not equal to the exact value of the population proportion
$p$, but it should be “close”. How close should we expect the sample
proportion to be to the actual value of $p$?

## <a name="17q2">Question 2</a>

------------------------------------------------------------------------

Answer each part below to construct a bootstrap distribution for the
sample proportion. Then use the result to answer the questions that
follow.

### <a name="17q2a">Question 2a</a>

------------------------------------------------------------------------

Complete the code cell below to construct a bootstrap distribution for
the sample proportion of observations with ozone concentration greater
than 70 ppb.

There are two operations to complete inside the for loop:

1.  Pick a bootstrap resample from the observed sample `nyc.oz`.
2.  Calculate the proportion of observations in the bootstrap resample
    with ozone concentration greater than 70 ppb.

#### <a name="17sol2a">Solution to Question 2a</a>

------------------------------------------------------------------------

Replace all six `??` in the code cell below with appropriate code. Then
run the completed code to generate a bootstrap distribution and mark the
observed sample proportion (in red) and the mean of the bootstrap
distribution (in blue) with vertical lines.

In [ ]:
#N <- 10^5  # Number of bootstrap samples
#boot.prop <- numeric(N)  # create vector to store bootstrap proportions

# for loop that creates bootstrap dist
#for (i in 1:N)
#{
#  x <- sample(??, size = ??, replace = ??)  # pick a bootstrap resample
#  boot.prop[i] <- ??  # compute bootstrap sample proportion
#}

# plot bootstrap distribution
#hist(boot.prop,  
#     breaks=20, 
#     xlab = "p-hat, sample proportion",
#     main = "Bootstrap Distribution for Sample Proportion")

# red line at the observed sample proportion
#abline(v = ??, col = "firebrick2", lwd = 2, lty = 1)

# blue line at the center of bootstrap dist
#abline(v = ??, col = "blue", lwd = 2, lty = 2)

### <a name="17q2b">Question 2b</a>

------------------------------------------------------------------------

Based on your answer to \[Question 2a\], calculate bootstrap estimate
for bias. *Note that answers will vary due to the randomness of
bootstrapping in \[Question 2a\].*

#### <a name="17sol2b">Solution to Question 2b</a>

------------------------------------------------------------------------

In [ ]:
# calculate bootstrap estimate of bias

<br>  
<br>

### <a name="17q2c">Question 2c</a>

------------------------------------------------------------------------

Based on your answer to \[Question 2a\], calculate the bootstrap
estimate for the standard error of the sampling distribution for sample
proportions. *Note that answers will vary due to the randomness of
bootstrapping in \[Question 2a\].*

#### <a name="17sol2c">Solution to Question 2c</a>

------------------------------------------------------------------------

In [ ]:
# calculate bootstrap standard error

<br>  
<br>

# <a name="17clt">Comparing Bootstrapping to CLT</a>

------------------------------------------------------------------------

Recall the [Central Limit Theorem (CLT) for
Proportions](11-Sampling-Dist-Prop.qmd),

$$\widehat{P} \sim N \left( \mu_{\hat{P}}, \mbox{SE}(\widehat{P}) \right) = \left( {\color{tomato}{p}}, \sqrt{\frac{{\color{tomato}{p}}(1-{\color{tomato}{p}})}{n}} \right).$$

The population proportion $p$ is unknown, but we can use the [plug-in
principle](16-Bootstrap-Dist.qmd#sec-plugin) and “plug-in” the sample
proportion ${\color{tomato}{\hat{p} = 0.2162}}$ in place of $p$ to
estimate the sampling distribution for the sample proportion:

$$\begin{aligned}
\widehat{P} \sim N \left( \mu_{\hat{P}}, \mbox{SE}(\widehat{P}) \right) &= \left( {\color{tomato}{p}}, \sqrt{\frac{{\color{tomato}{p}}(1-{\color{tomato}{p}})}{n}} \right) \\
& \approx N\left( {\color{tomato}{0.2162}}, \sqrt{\frac{{\color{tomato}{0.2162}}(1-{\color{tomato}{0.2162}})}{111}} \right) \\
& \approx N( 0.2162, 0.0391).
\end{aligned}$$

| Method                             | Mean of Sampling Distribution | Standard Error |
|-------------------------------|---------------------------|--------------|
| Bootstrap[1]                       | $0.2164$                      | $0.0390$       |
| CLT with $\hat{p}$ in place of $p$ | $0.2162$                      | $0.0391$       |

The two methods, bootstrap distribution and the estimate from using the
CLT, give us consistent results!

# <a name="17point-int">Point and Interval Estimates</a>

------------------------------------------------------------------------

Estimators obtained using maximum likelihood estimates (MLE), method of
moments (MoM), or plug-in principle are examples of
<span style="color:tomato">**point estimates**</span> since they result
in a <span style="color:tomato">**single value**</span> to estimate the
unknown population parameter.

-   For example, using the plug-in principle, we have a point estimate
    ${\color{tomato}{p \approx \hat{p} = 0.2162}}$.
-   Since there is variability from sample to sample, it is not very
    likely that our point estimate is perfectly accurate, but it is
    likely close.
-   We get a very specific estimate that is unlikely to be exactly equal
    to the actual parameter value.
-   <span style="color:tomato">**This is like trying to catch a fish
    with a spear.**</span>

Usually when estimating an unknown population parameter, we give an
<span style="color:dodgerblue">**interval estimate**</span> that gives
<span style="color:dodgerblue">**range of plausible values**</span> for
the parameter by accounting for the uncertainty due to the variability
in sampling.

-   Using both mean and standard error from the bootstrap distribution
    in \[Question 2c\], one possible interval estimate is
    $$p \approx \hat{p}_{\rm boot} \pm \mbox{SE}_{\rm boot}  = 0.216 \pm 0.039.$$
    -   <span style="color:dodgerblue">An interval of plausible values
        for $p$ is from $0.177$ to $0.255$</span>.  
-   It is more likely the actual value of the parameter is somewhere
    inside the interval estimate than it is to be exactly equal to a
    point estimate.
-   However, interval estimate are less precise.
-   <span style="color:dodgerblue">**This is like trying to catch a fish
    with a net.**</span>
-   We are more likely to catch a fish even if we do not know
    specifically where the fish is located!
-   <span style="color:dodgerblue">**We frequently estimate parameters
    with an interval to account for the variability of sample
    statistics.**</span>

<figure>
<img
src="https://i0.wp.com/pixahive.com/wp-content/uploads/2020/09/Man-Throwing-a-Fishing-Net-79117-pixahive.jpg?fit=778%2C518&amp;ssl=1"
data-fig-alt="Man Throwing a Fishing Net"
alt="Credit: Rahil, Public Domain, Pixahive" />
<figcaption aria-hidden="true">Credit: Rahil, <a
href="https://creativecommons.org/share-your-work/public-domain/cc0/">Public
Domain</a>, <a
href="https://pixahive.com/photo/man-throwing-a-fishing-net/">Pixahive</a></figcaption>
</figure>

<figure>
<img
src="https://cdn2.picryl.com/photo/1898/12/31/fmib-48360-indian-sealing-canoe-showing-method-of-throwing-spear-7cbd5a-1024.jpeg"
data-fig-alt="Sealing Canoe Showing Method of Throwing Spear"
alt="Credit: Townsend, C. H. (1898), Public Domain" />
<figcaption aria-hidden="true">Credit: <a
href="https://garystockbridge617.getarchive.net/amp/media/fmib-48360-indian-sealing-canoe-showing-method-of-throwing-spear-7cbd5a">Townsend,
C. H</a>. (1898), Public Domain</figcaption>
</figure>

Fishing with an interval estimate compared to a point estimate.

# <a name="17ci-95">A 95% Bootstrap Percentile Confidence Interval</a>

------------------------------------------------------------------------

The interval between the $2.5$ and $97.5$ percentiles (or $0.025$ and
$0.975$ quantiles) of the bootstrap distribution of a statistic is a
<span style="color:dodgerblue">**95% bootstrap percentile confidence
interval**</span> for the corresponding parameter.

-   If most of the sample statistics are located in a certain interval
    of the bootstrap distribution, it seems plausible the true value of
    the parameter is in this interval!
-   We would say we are 95% confident that the interval contains the
    actual value of the population parameter since 95% of the bootstrap
    resamples are inside this interval.
-   <span style="color:dodgerblue">**Confidence interval estimates give
    a range of plausible values for an unknown parameter.**</span>

## <a name="17q3">Question 3</a>

------------------------------------------------------------------------

In \[Question 2\], we created a bootstrap distribution (stored in the
vector `boot.prop`) for the sample proportion of times the ozone
concentration exceeds 70 ppb. One possible bootstrap distribution is
plotted below. *Note that bootstrap distributions will vary slightly
depending on the 100,000 bootstrap resamples that are randomly
selected.*

[1] Bootstrap estimates from \[Question 2\] will slightly vary.

In [ ]:
# plot bootstrap distribution
hist(boot.prop,  
     breaks=20, 
     xlab = "p-hat, sample proportion",
     main = "Bootstrap Distribution for Sample Proportion")

# red line at the observed sample proportion
abline(v = mean(nyc.oz > 70), col = "firebrick2", lwd = 2, lty = 1)

# blue line at the center of bootstrap dist
abline(v = mean(boot.prop), col = "blue", lwd = 2, lty = 2)

### <a name="17q3a">Question 3a</a>

------------------------------------------------------------------------

Using the bootstrap statistics stored in `boot.prop`, find the lower and
upper cutoffs for a 95% bootstrap percentile confidence interval for the
proportion of all times the ozone concentration in NYC exceeds 70 ppb.

Recall the `quantile()` function in R. Run the command `?quantile` for a
refresher!

#### <a name="17sol3a">Solution to Question 3a</a>

------------------------------------------------------------------------

Replace all four `??` in the code cell below with appropriate code. Then
run the completed code to compute lower and upper cutoffs for a 95%
bootstrap percentile confidence interval.

In [ ]:
# find cutoffs for 95% bootstrap CI
#lower.boot.95 <- quantile(??, probs = ??)  # find lower cutoff
#upper.boot.95 <- quantile(??, probs = ??)  # find upper cutoff

# print values to screen
#lower.boot.95
#upper.boot.95

**Based on the output above, a 95% bootstrap percentile confidence
interval is from ?? to ??.**

<br>  
<br>

### <a name="17q3b">Question 3b</a>

------------------------------------------------------------------------

The code cell below plots a bootstrap distribution corresponding of the
sample proportions stored in `boot.prop` along with two blue vertical
lines to mark the lower and upper cutoffs for a 95% bootstrap percentile
confidence interval. A red vertical line marks the value of the sample
proportion we calculated from the original sample.

Run the code cell below to illustrate the confidence interval on the
bootstrap distribution. There is nothing to edit in the code cell. Then
in the space below, explain the practical meaning of the interval to a
person with little to no background in statistics.

In [ ]:
#################################
# code is ready to run!
# no need to edit the code cell
#################################

# plot bootstrap distribution
hist(boot.prop,  
     breaks=20, 
     xlab = "p-hat, sample proportion",
     main = "Bootstrap Distribution for Sample Proportion")

# red line at the observed sample proportion
abline(v = mean(nyc.oz > 70), col = "firebrick2", lwd = 2, lty = 1)

# blue lines marking cutoffs
abline(v = lower.boot.95, col = "blue", lwd = 2, lty = 2)
abline(v = upper.boot.95, col = "blue", lwd = 2, lty = 2)

#### <a name="17sol3b">Solution to Question 3b</a>

------------------------------------------------------------------------

Interpret the confidence interval from \[Question 3a\].

<br>  
<br>  
<br>

### <a name="17q3c">Question 3c</a>

------------------------------------------------------------------------

Sometimes, it is desirable to describe the interval as some value plus
or minus some margin of error. Construct a
<span style="color:dodgerblue">**symmetric 95% bootstrap confidence
interval**</span> for the proportion of time the ozone concentration in
NYC exceeds 70 ppb. Compare the symmetric confidence interval to your
percentile confidence interval in \[Question 3a\].

Recall for normal distributions, approximately 95% of the data is within
2 standard deviations of center of the distribution.

#### <a name="17sol3c">Solution to Question 3c</a>

------------------------------------------------------------------------

Replace each `??` in the code cell below with appropriate code.

In [ ]:
#?? - 2*??  # going 2 SE's below the observed sample proportion
#?? + 2*??  # going 2 SE's above the observed sample proportion

**Based on the output above, a symmetric 95% bootstrap confidence
interval is from ?? to ??.**

<br>  
<br>

# <a name="17adjust-cl">Adjusting the Confidence Level</a>

------------------------------------------------------------------------

We would expect the actual value of the unknown population parameter to
equal the corresponding statistic calculated from one of the 100,000
bootstrap resamples in our distribution. Since 95% of the bootstrap
statistics are inside the confidence interval:

> **A 95% bootstrap confidence interval has about a 95% chance of
> containing the actual value of the population parameter.**

The <span style="color:dodgerblue">**confidence level**</span> is the
success rate of success of the interval estimate. We can choose
different confidence levels for an interval estimate.

-   A 95% confidence level is most common.
-   Other common confidence levels are 80%, 90%, 99% and 99.9%.
-   We choose a confidence level. It is not something we calculate.
-   **What happens to an interval estimate when we change the confidence
    level?**

## <a name="17q4">Question 4</a>

------------------------------------------------------------------------

In \[Question 2\], we created a bootstrap distribution (stored in the
vector `boot.prop`) for the sample proportion of times the ozone
concentration exceeds 70 ppb. In \[Question 3\], we used the bootstrap
distribution to construct a 95% bootstrap percentile confidence
interval. In this question, we will investigate what happens when we
change the confidence level.

### <a name="17q4a">Question 4a</a>

------------------------------------------------------------------------

Complete the first code cell below to give a 90% bootstrap percentile
confidence interval to estimate the proportion of all time in NYC when
the ozone concentration exceeds 70 ppb.

Then complete the second code cell to plot a histogram of the bootstrap
distribution from \[Question 2\] with the upper and lower 90% bootstrap
confidence interval cutoffs marked with vertical lines similar to the
plot in \[Question 3b\].

#### <a name="17sol4a">Solution to Question 4a</a>

------------------------------------------------------------------------

**Based on the output below, a 90% bootstrap percentile confidence
interval is from ?? to ??.**

<br>  
<br>

Replace all four `??` in the code cell below with appropriate code. Then
run the completed code to compute lower and upper cutoffs for a 90%
bootstrap percentile confidence interval.

In [ ]:
# find cutoffs for 90% bootstrap CI
#lower.prop.90 <- quantile(??, probs = ??)  # find lower cutoff
#upper.prop.90 <- quantile(??, probs = ??)  # find upper cutoff

# print to screen
#lower.prop.90
#upper.prop.90

Nothing to edit in the code cell below. Just be sure you first run the
code cell above to calculate and store the cutoffs `lower.prop.90` and
`upper.prop.90`.

In [ ]:
##################################
# code is ready to run!
# no need to edit the code cell
##################################

# plot bootstrap distribution
hist(boot.prop,  
     breaks=20, 
     xlab = "p-hat, sample proportion",
     main = "Bootstrap Distribution for Sample Proportion")

# red line at the observed sample proportion
abline(v = mean(nyc.oz > 70), col = "firebrick2", lwd = 2, lty = 1)

In [ ]:
# blue lines marking cutoffs
#abline(v = lower.prop.90, col = "blue", lwd = 2, lty = 2)
#abline(v = upper.prop.90, col = "blue", lwd = 2, lty = 2)

### <a name="17q4b">Question 4b</a>

------------------------------------------------------------------------

Complete the first code cell below to give a 99% bootstrap percentile
confidence interval to estimate the proportion of all time in NYC when
the ozone concentration exceeds 70 ppb.

Then complete the second code cell to plot a histogram of the bootstrap
distribution from \[Question 2\] with the upper and lower 99% bootstrap
confidence interval cutoffs marked with vertical lines similar to the
plot in \[Question 3b\].

#### <a name="17sol4b">Solution to Question 4b</a>

------------------------------------------------------------------------

**Based on the output below, a 99% bootstrap percentile confidence
interval is from ?? to ??.**

<br>  
<br>

Replace all four `??` in the code cell below with appropriate code. Then
run the completed code to compute lower and upper cutoffs for a 90%
bootstrap percentile confidence interval.

In [ ]:
# find cutoffs for 99% bootstrap CI
#lower.prop.99 <- quantile(??, probs = ??)  # find lower cutoff
#upper.prop.99 <- quantile(??, probs = ??)  # find upper cutoff

# print to screen
#lower.prop.99
#upper.prop.99

Nothing to edit in the code cell below. Just be sure you first run the
code cell above to calculate and store the cutoffs `lower.prop.99` and
`upper.prop.99`.

In [ ]:
##################################
# code is ready to run!
# no need to edit the code cell
##################################

# plot bootstrap distribution
hist(boot.prop,  
     breaks=20, 
     xlab = "p-hat, sample proportion",
     main = "Bootstrap Distribution for Sample Proportion")

# red line at the observed sample proportion
abline(v = mean(nyc.oz > 70), col = "firebrick2", lwd = 2, lty = 1)

In [ ]:
# blue lines marking cutoffs
#abline(v = lower.prop.99, col = "blue", lwd = 2, lty = 2)
#abline(v = upper.prop.99, col = "blue", lwd = 2, lty = 2)

### <a name="17q4c">Question 4c</a>

------------------------------------------------------------------------

When we decreased the confidence level from 95% in \[Question 3a\] to
90% in \[Question 4a\], did the interval estimate get wider or more
narrow? Explain why this makes practical sense. Feel free to explain
using the [fishing analogy](#sec-fishing) from earlier.

#### <a name="17sol4c">Solution to Question 4c</a>

------------------------------------------------------------------------

<br>  
<br>  
<br>

### <a name="17q4d">Question 4d</a>

------------------------------------------------------------------------

Explain the trade-off between choosing a confidence level and the
precision of the interval estimate. In particular, why would choose a
95% confidence interval over an interval that has a greater chance of
success, such as a 99% confidence interval?

#### <a name="17sol4d">Solution to Question 4d</a>

------------------------------------------------------------------------

<br>  
<br>  
<br>

# <a name="17ind-pop">Comparing Two Independent Populations</a>

------------------------------------------------------------------------

<figure>
<img
src="https://static01.nyt.com/images/2020/01/28/well/physed-babies/physed-babies-jumbo.jpg?quality=75&amp;auto=webp"
style="width:50.0%" data-fig-align="left" data-fig-alt="Toddlers"
alt="Credit: iStock" />
<figcaption aria-hidden="true">Credit: iStock</figcaption>
</figure>

Thus far, we have been bootstrapping from a sample that has just one
variable of interest. For example, in `nyc.oz` we have $n=111$ observed
ozone concentrations in NYC. For each observation in our sample, we have
one variable of interest, namely the ozone concentration (in ppb). In
statistics, we are often interested in determining whether there is an
association between different two or more variables. For example, we can
ask:

> Does smoking during pregnancy have an affect on the weight of the baby
> at birth?

To answer this question, we can collect data from a random sample of
newborn babies. We would need to collect more than one variable for each
newborn in our sample.

## <a name="17q5">Question 5</a>

------------------------------------------------------------------------

Explain how you could design a study to collect data that could help
determine whether smoking during pregnancy has an affect on the weight
of the baby at birth.

### <a name="17sol5">Solution to Question 5</a>

------------------------------------------------------------------------

<br>  
<br>  
<br>

# <a name="17smoke-wt">Case Study: Smoking and Birth Weights</a>

------------------------------------------------------------------------

To help explore this question, we will use the sample in the data
frame[1] `birthwt` that is in the package `MASS` which should already be
installed. In the code cell below, the `MASS` package is loaded and
numerical summaries for all variables in `birthwt` are computed and
displayed.

If you receive an error message when running the code cell below, then
you do not have the `MASS` package installed.

-   In the R Console, run the command `install.packages("MASS")`.
-   Then rerun the code cell below.

## <a name="17load-wt">Loading the Birth Weight Sample</a>

------------------------------------------------------------------------

[1] Baystate Medical Center, Springfield, Massachusetts

In [ ]:
library(MASS)  # load MASS package
summary(birthwt)  # summary of data frame

## <a name="17q6">Question 6</a>

------------------------------------------------------------------------

How many observations are in the data set? How many variables? Which
variables are categorical and which are quantitative? Which variables
are most important in helping determine whether smoking during pregnancy
has an affect on the weight of the baby at birth?

### <a name="17sol6">Solution to Question 6</a>

------------------------------------------------------------------------

<br>  
<br>  
<br>

## <a name="17clean-wt">Cleaning the Birth Weight Data</a>

------------------------------------------------------------------------

The variable `smoke` is being stored as a quantitative variable.

-   Pregnant parents that did not smoke have a `smoke` value equal to
    `0`.
-   Pregnant parents that were smokers have a `smoke` value equal to
    `1`.
-   Run the code cell below to make these categories more clearly
    labeled.
    -   Non-smokers are assigned the character string `"no"`.
    -   Smokers are assigned the character string `"smoker"`.
    -   We use the `factor()` command to convert the `smoke` variable to
        a categorical variable.
    -   The output tells us out of 189 parents, 115 self-identified as
        non-smokers and 74 as smokers.

In [ ]:
birthwt$smoke[birthwt$smoke == 0]  <- "no"
birthwt$smoke[birthwt$smoke == 1]  <- "smoker"
birthwt$smoke <- factor(birthwt$smoke)
summary(birthwt$smoke)

## <a name="17q7">Question 7</a>

------------------------------------------------------------------------

Complete the code cell below to create a side-by-side box plots to
compare the distribution of weights for smokers and non-smokers.

#### <a name="17sol7">Solution to Question 7</a>

------------------------------------------------------------------------

Replace each `??` in the code cell below to generate side-by-side box
plots for comparison.

In [ ]:
# create side by side box plots
#plot(?? ~ ??, data = ??,
#     col = c("springgreen4", "firebrick2"),
#     main = "Comparison of Birth Weights from Smokers and Non-Smokers",
#     xlab = "Smoking Status of Pregnant Parent",
#     ylab = "Birth Weight (in grams)",
#     names = c("Non-smoker", "Smoker"))

<br>  
<br>

## <a name="17-diff-means">Difference in Two Independent Means</a>

------------------------------------------------------------------------

Our statistical question is:

> Does smoking during pregnancy have an affect on the weight of the baby
> at birth?

In this example, we have two independent populations of parents to
consider.

-   All pregnant parents that did not smoke during pregnancy.
-   All pregnant parents that smoked during pregnancy.
-   A pregnant person is either in one group or the other, but not both.

Ideally, if we had access to data on every baby that has ever been born,
then we could:

-   Calculate $\mu_{\rm{non}}$, the mean birth weight of *all* children
    of non-smokers.
-   Calculate $\mu_{\rm{smoker}}$, the mean birth weight of *all*
    children of smokers.
-   Consider how large is the difference in the two means,
    $\mu_{\rm{non}} - \mu_{\rm{smoker}}$.
    -   If the difference in populations means is 0, there is no
        difference in mean birth weights.
    -   If the difference is not 0, then there is a difference in mean
        birth weights.

Although we have data from a random sample rather than the entire
population, we can perform equivalent analysis comparing two sample
means:

-   Calculate $\bar{x}_{\rm{non}}$, the *sample* mean birth weight of
    children of non-smokers in the sample.
-   Calculate $\bar{x}_{\rm{smoker}}$, the *sample* mean birth weight of
    children of smokers in the sample.
-   Consider how large is the difference in the two means,
    $\bar{x}_{\rm{non}} - \bar{x}_{\rm{smoker}}$.
    -   If the difference is *close to zero*, there is likely no
        difference in the population means.
    -   If the difference is *not close to zero*, there likely is a
        difference in the population means.
    -   How can we determine what we mean by “close to zero”?

## <a name="17subset">Subsetting Into Two Independent Samples</a>

------------------------------------------------------------------------

At the moment, we have all of the sample data for smokers and
non-smokers stored in the same data frame named `birthwt`. How can we
calculate the sample mean birth weights for the non-smoking group
separate from the smoking group? One way to compare the two samples is
to split the sample into two independent subsets based on whether or not
the child was birthed by a smoker or not.

## <a name="17q8">Question 8</a>

------------------------------------------------------------------------

Answer the questions to find an initial estimate for the difference in
the mean birth weights of all children born to a non-smoking parent
compared to the mean birth weight of all children born to a parent that
did smoke while pregnant.

### <a name="17q8a">Question 8a</a>

------------------------------------------------------------------------

Complete each of the `subset()` commands below to subset the data into
two independent samples: parent was a smoker and parent was a
non-smoker.

#### <a name="17sol8a">Solution to Question 8a</a>

------------------------------------------------------------------------

In [ ]:
# subset the sample into two independent samples
#non <- subset(??, smoke == ??)
#smoker <- subset(??, smoke == ??)

<br>  
<br>

### <a name="17q8b">Question 8b</a>

------------------------------------------------------------------------

Complete the code cell below to calculate, store, and print the
difference in sample means based on the data in our original sample.

#### <a name="17sol8b">Solution to Question 8b</a>

------------------------------------------------------------------------

In [ ]:
# calculate difference in sample means
#obs.diff <- ??
#obs.diff  # print observed difference to screen

<br>  
<br>

### <a name="17q8c">Question 8c</a>

------------------------------------------------------------------------

Based on your answer to \[Question 8b\], given a **point estimate** for
the difference in the mean birth weights of all children born to a
non-smoking parent compared to the mean birth weight of all children
born to a parent that did smoke while pregnant. *Include units in your
answer.*

#### <a name="17sol8c">Solution to Question 8c</a>

------------------------------------------------------------------------

<br>  
<br>  
<br>

### <a name="17q8d">Question 8d</a>

------------------------------------------------------------------------

Based on your estimate in \[Question 8c\], do you believe there is a
difference in the mean birth weight of all babies whose parent smoked
while pregnant compared to the mean birth weight of all babies whose
parent did not smoke while pregnant?

#### <a name="17sol8d">Solution to Question 8d</a>

------------------------------------------------------------------------

<br>  
<br>  
<br>

## <a name="17uncertain">Accounting for Uncertainty in Sampling</a>

------------------------------------------------------------------------

In the case of comparing samples, we need to be mindful of the
randomness involved in the sampling process. If we pick another sample
of 189 babies and compare the difference in sample means, we will likely
get a different estimate for the difference in mean birth weights. **How
can we determine whether or not the point estimate in \[Question 8c\] is
“close enough to zero”?**

# <a name="17boot-two">Bootstrapping Two Independent Samples</a>

------------------------------------------------------------------------

Given independent samples of sizes $m$ and $n$ from two independent
populations:

1.  Draw a bootstrap resample of size $m$ *with replacement* from the
    first sample.

In [ ]:
m.non <- length(non$bwt)  # m, size of sample 1
temp.non <- sample(non$bwt, size = m.non, replace = TRUE)

1.  Draw a bootstrap resample of size $n$ *with replacement* from the
    second sample.

In [ ]:
n.smoker <- length(smoker$bwt)  # n, size of sample 2
temp.smoker <- sample(smoker$bwt, size = n.smoker, replace = TRUE)

1.  Compute a statistic that compares the two groups such as a
    difference or ratio of two statistics (means, proportions,
    variances, etc.)

In [ ]:
diff.resample <- mean(temp.non) - mean(temp.smoker)
diff.resample

1.  Repeat this process many (100,000) times.
2.  Construct a bootstrap distribution of the comparison statistics from
    each resample.

## <a name="17q9">Question 9</a>

------------------------------------------------------------------------

Follow the steps below to generate a bootstrap distribution for the
difference in sample mean birth weights for babies born to non-smokers
compared to babies born to smokers. Then use the bootstrap distribution
to obtain a 95% bootstrap percentile confidence interval for the
corresponding difference in population means.

### <a name="17q9a">Question 9a</a>

------------------------------------------------------------------------

Complete the code cell below to construct a bootstrap distribution for
the difference in the sample mean birth weights of of babies born to
non-smokers compared to smokers.

#### <a name="17sol9a">Solution to Question 9a</a>

------------------------------------------------------------------------

Replace all nine `??` in the code cell below with appropriate code. Then
run the completed code to generate a bootstrap distribution and mark the
observed sample proportion (in red) and the mean of the bootstrap
distribution (in blue) with vertical lines.

In [ ]:
#N <- 10^5  # Number of bootstrap samples
#boot.diff.mean <- numeric(N)  # create vector to store bootstrap proportions

# for loop that creates bootstrap dist
#for (i in 1:N)
#{
#  x.non <- sample(??, size = ??, replace = ??)  # pick a bootstrap resample
#  x.smoker <- sample(??, size = ??, replace = ??)  # pick a bootstrap resample
#  boot.diff.mean[i] <- ??  # compute difference in sample means
#}

# plot bootstrap distribution
#hist(boot.diff.mean,  
#     breaks=20, 
#     xlab = "x.bar.non - x.bar.smoker (in grams)",
#     main = "Bootstrap Distribution for Difference in Means")

# red line at the observed  difference in sample means
#abline(v = ??, col = "firebrick2", lwd = 2, lty = 1)

# blue line at the center of bootstrap dist
#abline(v = ??, col = "blue", lwd = 2, lty = 2)

### <a name="17q9b">Question 9b</a>

------------------------------------------------------------------------

Complete the code cell below to give a 95% bootstrap percentile
confidence interval to estimate the difference in the mean birth weight
of all babies born to non-smokers compared the to mean birth of all
babies born to smokers.

#### <a name="17sol9b">Solution to Question 9b</a>

------------------------------------------------------------------------

**Based on the output below, a 95% bootstrap percentile confidence
interval is from ?? to ??.**

<br>  
<br>

In [ ]:
# find cutoffs for 95% bootstrap CI
#lower.bwt.95 <- quantile(??, probs = ??)  # find lower cutoff
#upper.bwt.95 <- quantile(??, probs = ??)  # find upper cutoff

# print to screen
#lower.bwt.95
#upper.bwt.95

### <a name="17q9c">Question 9c</a>

------------------------------------------------------------------------

Interpret the practical meaning of your interval estimate in \[Question
9b\]. Do you think it is plausible to conclude smoking does have an
effect on the weight of a newborn? Explain why or why not.

#### <a name="17sol9c">Solution to Question 9c</a>

------------------------------------------------------------------------

<br>  
<br>  
<br>

------------------------------------------------------------------------

![Creative Commons
License](https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png) <nbsp>

*Statistical Methods: Exploring the Uncertain* by [Adam
Spiegler](https://github.com/CU-Denver-MathStats-OER/Statistical-Theory)
is licensed under a [Creative Commons
Attribution-NonCommercial-ShareAlike 4.0 International
License](http://creativecommons.org/licenses/by-nc-sa/4.0/).